In [2]:
# !pip install -q qdrant_client

In [1]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import os

c:\Users\91790\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [ ]:
client = QdrantClient(
    "https://e06f23a0-bcc3-4428-ac01-07e17fa4b92f.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key="API"
)

In [4]:
client

In [5]:
encoder = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
client.create_collection(
    collection_name="furrbot",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE, 
        on_disk=True
    )
)

True

In [7]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='furrbot')])

In [8]:
import os 

file_paths = [f for f in os.listdir("vector_database/text_data") if f.endswith('.json')]

In [10]:
# add folder path to file_paths
file_paths = ["vector_database/text_data/" + f for f in file_paths]

In [12]:
file_paths[:4]

['vector_database/text_data/Bees.json',
 'vector_database/text_data/Birds.json',
 'vector_database/text_data/Cats.json',
 'vector_database/text_data/Chicken and Ducks.json']

In [14]:
## get length of points 
import json

# add all these lists to be a single list 
for item in file_paths:
    with open(item) as f:
        data = json.load(f)
    print(f"length of points in {data[0]['pet']} is {len(data)}")

length of points in Bees is 8
length of points in Birds is 884
length of points in Cats is 966
length of points in Chicken and Ducks is 107
length of points in Chinchillas is 268
length of points in Dogs is 4949
length of points in Ferrets is 495
length of points in Fish is 666
length of points in Gerbils is 137
length of points in Guinea Pigs is 353
length of points in Hamsters is 339
length of points in Horses is 714
length of points in Pigs is 13
length of points in Prairie Dogs is 79
length of points in Rabbits is 842
length of points in Rats is 256
length of points in Reptiles is 119
length of points in Sugar Gliders is 31


In [15]:
text_list = []

for item in file_paths:
    with open(item) as f:
        data = json.load(f) 
    text_list += data

In [19]:
text_list[:3]

[{'text': 'beekeeping 101 | petmd petmd.com /farm-animal/beekeeping-101 there are 17,000 species of bees in the world and around 80 million beehives. there are 115-125,000 beekeepers in the united states and beekeeping has been around since 15,000 bc. honeybees are integral to agriculture and crops. a third of the diet in the u.s. comes from insect-pollinated plants alone and 80% of those crops are pollinated by honeybees. these crops include blueberries, cranberries, apples, pumpkins, and peaches. 90% of all wild plants depend on animal pollination. without bees, these crops would not thrive. there are other pollinators, like insects, birds, and bats, but honeybees can be moved wherever they need to pollinate. in addition, without honeybees, it is costly to produce certain fruits, legumes, vegetables, nuts, and seeds.',
  'topic': 'Beekeeping 101 | PetMD',
  'blog_name': 'Beekeeping 101 _ PetMD',
  'pet': 'Bees',
  'word_count': 125,
  'chunk_size': 162},
 {'text': 'keeping bees there

In [22]:
client.upload_points(
    collection_name='furrbot', 
    points=[
        models.PointStruct(
            id=idx, 
            vector=encoder.encode(doc['text']).tolist(),
            payload=doc
        )
        for idx, doc in enumerate(text_list)
    ]
)

In [23]:
# Let's query it 
hits = client.query_points(
    collection_name='furrbot',
    query=encoder.encode("What to feed a persian Cat?").tolist(),
    limit=2, 
    query_filter=models.Filter(
                must=[
                    models.FieldCondition(
                        key="word_count",
                        range=models.Range(
                            gte=40,
                            lte=300
                        )
                    )
                ]
            )
    ).points

In [24]:
hits

[ScoredPoint(id=1513, version=23, score=0.7844912, payload={'text': "what to feed a persian cat persians aren’t the most active cats, and care must be taken to prevent them from carrying excess weight. “some people refer to them as lazy, but they just enjoy sleeping and sitting calmly,” says stephen quandt, a certified feline training and behavior specialist (cftbs). to ensure your cat's nutrition needs are met, choose complete and balanced food that meets the association of american feed control officials (aafco) recommendations for their life stage. then, talk to your veterinarian about the number of calories your cat should consume, based on their lifestyle.", 'topic': 'What To Feed a Persian Cat', 'blog_name': 'Cats_Breeds_Persian', 'pet': 'Cats', 'word_count': 96, 'chunk_size': 112}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1515, version=23, score=0.7333615, payload={'text': 'how much should you feed a persian? persian cats should be fed the appropriate numb